# 简介

本项目以CUB 200 2011数据集为样例（以下简称CUB），实现了大作业的简单框架，仅作抛砖引玉，欢迎同学们查阅相关资料深入学习，并在课上课下提出讨论。

若在个人电脑上进行实验，需要配置相关环境。Pycharm、Anaconda和pytorch安装教程可参考https://blog.csdn.net/Bluebro/article/details/127161165 。注意安装pytorch时需要确定时cpu版本还是cuda版本，若安装cuda版本的pytorch则要求先安装英伟达cuda驱动（可参考https://blog.csdn.net/chen565884393/article/details/127905428）

# 目录

1.项目组成

2.数据预处理

3.项目代码运行

4.模型输出以及查看和修改

5.可解释性

6.可进一步提升

7.选做


# 1.项目组成

本项目文件由以下结构组成

|-- DL2025_proj/

    |-- analysis/ 模型结果输出的文件夹，以及数据统计工具类
    
        |-- result/ 最终结果输出文件夹
        
            |-- xxx/ 按版本号输出的文件夹
            
                |-- foldk.csv 第k折交叉验证的真实标签'true'，预测标签'pred'，预测概率'prob'，图像路径'path'
                
                |-- foldk_report.csv 第k折交叉验证的分类报告
                
                |-- foldk_fc_weight.npy 第k折交叉验证的全连接层权重
        
        |-- mid_feature/ 钩子（hook）获取的中间结果（某层的输入输出特征）输出文件夹
        
            |-- xxx/ 按版本号输出的文件夹
            
                |-- foldk/ 按第k折交叉验证输出的文件夹
                
                    |-- xxx 逐个输出测试集样本的中间结果
                    
        |-- analysis_tools.py 绘出混淆矩阵、roc曲线、计算CAM、保存热力图的类
        
        |-- statistic_result.py 统计评价指标和auc的类
                
    |-- ckpt/ 模型参数输出的文件夹
    
        |-- xxx/ 按版本号输出的文件夹
        
            |-- foldk/ 按第k折交叉验证输出的文件夹
            
                |-- epoch=xxx.pth 模型参数输出，包含迭代次数'epoch'，保存路径'save_dir'，模型参数字典'state_dict'，优化器参数字典'optimizer'
        
    |-- converter/ 文件读取的工具类的文件夹
            
    |-- csv_file/ csv格式的数据集目录的文件夹
                
    |-- data_utils/ 数据读取的工具类的文件夹
    
        |-- data_loader.py 数据读取
    
        |-- transform.py 数据转换和数据增强
            
    |-- datasets/ 数据集的文件夹
    
        |-- CUB_200_2011/ CUB 200 2011数据集
        
            |-- attributes.txt 
            
            |-- CUB_200_2011/ 
            
                |-- attributes 
                
                |-- images 图像数据
                
                |-- parts 
                
                |-- bounding_boxes.txt 
                
                |-- classes.txt.txt 
                
                |-- image_class_labels.txt 
                
                |-- images.txt 目录文件
                
                |-- README
                
                |-- train_test_split.txt 训练集和测试集的划分文件

    |-- gen_dataset/ 生成模型生成的数据集的文件夹
                    
    |-- log/ 模型运行的过程数据输出的文件夹
    
        |-- xxx/ 按版本号输出的文件夹
        
            |-- foldk/ 按第k折交叉验证输出的文件夹
            
                |-- data/ 过程数据输出文件夹
                
                |-- events.xxxxx 命令行输出结果
        
    |-- model/ 模型文件夹
                
    |-- config.py 模型配置
    
    |-- main_GAN.py 简单的GAN模型的入口
    
    |-- main_interpretability.py 简单的利用CAM保存热力图的入口
    
    |-- main_VLM.py 简单的VLM模型的入口
    
    |-- main.py 项目模型入口
    
    |-- Readme.ipynb 项目的说明
    
    |-- requirements.txt 依赖库
    
    |-- make_csv.py 制作csv格式的数据集目录的函数
    
    |-- trainer.py 训练器，训练过程主要代码
    
    |-- utils.py 工具类
    
# 2.数据预处理

1.到数据集官网上阅读数据集相关信息，并将数据数据集下载到datasets文件夹中，至少**包含图像数据，索引文件，训练集和测试集的划分文件**，将数据放到datasets/CUB_200_2011路径下。

2.考虑到不同数据集的结构不同，本项目使用统一建立的csv文件作为索引管理各个数据集，这类文件结构如下：第一列“id”标识数据的路径（项目根目录的相对路径，或者绝对路径），第二列“label”标识数据的实际类别，训练集和测试集分别用不同的csv文件作为索引。

3.修改make_csv.py，为不同数据集编写函数，利用索引文件和划分文件，生成对应数据集的csv索引文件和训练集、测试集的csv索引文件到目录.csv_flie/中（已为CUB编写代码并生成csv索引）。

# 3.项目代码运行

## 修改超参数和路径

1.修改config.py，指定网络名称NET_NAME，版本VERSION，**版本决定数据保存的文件夹名，相同版本号会覆盖保存的文件**，以及超参数INIT_TRAINER和SETUP_TRAINER。其中INIT_TRAINER前6个常用参数可以在bash运行时覆盖（详见下文）,具体参数释义如下：
~~~python
INIT_TRAINER = {
    'net_name':NET_NAME, #网络名称，由NET_NAME决定
    'lr':1e-4,  #初始学习率
    'n_epoch':50, #迭代次数
    'num_classes':200, #分类的类别别数
    'image_size':224, #图像的边长
    'batch_size':16, #批次大小
    'train_mean':CUB_TRAIN_MEAN, #训练集的均值
    'train_std':CUB_TRAIN_STD, #训练集的标准差
    'num_workers':2, #dataloader的线程数
    'device':DEVICE, #运行使用的设备
    'pre_trained':PRE_TRAINED, #是否载入模型预训练权重，由PRE_TRAINED决定
    'weight_decay': 1e-4, #优化器L2正则化的λ参数
    'momentum': 0.9, #SGD优化器的参数
    'gamma': 0.1, #MultiStepLR学习率策略的参数
    'milestones': [20,40], #MultiStepLR学习率策略的参数
    'T_max':5, #CosineAnnealingLR学习率策略的参数
    'use_fp16':True #use_fp16是否使用半精度训练
 }

SETUP_TRAINER = {
    'output_dir':'./ckpt/{}'.format(VERSION), #模型权重输出路径
    'log_dir':'./log/{}'.format(VERSION), #模型过程输出路径
    'optimizer':'Adam', #优化器
    'loss_fun':'Cross_Entropy', #损失函数选择
    'class_weight':None, #损失函数的类权重，对于类别不平衡数据有用
    'lr_scheduler':'MultiStepLR' #学习率策略选择
}
~~~

2.修改main.py，指定所选数据集的csv索引文件路径，如下：

~~~python
if 'train' in args.mode:
    csv_path = './csv_file/cub_200_2011.csv_train.csv'
...
        
elif 'inf' in args.mode:
    test_csv_path = './csv_file/cub_200_2011.csv_test.csv'

~~~

3.进行训练时，config.py中PRE_TRAINED参数设置为False时，将从头开始训练；PRE_TRAINED参数设置为True时，将在线下载预训练模型参数（具体模型可看model中resnet、vit文件开头指定的url路径）。

4.进行推理时，config.py中WEIGHT_PATH（单折推理）和WEIGHT_PATH_LIST（多折推理）指定了保存的模型权重路径（只需提供对应version和fold）。

## 运行程序

单折、多折训练
~~~bash
python main.py -m train
python main.py -m train-cross
~~~

单折、多折推理
~~~bash
python main.py -m inf
python main.py -m inf-cross
~~~

单折、多折推理（不需要结果输出）
~~~bash
python main.py -m inf -s n
python main.py -m inf-cross -s n
~~~

config.py中的INIT_TRAINER前6个常用参数可以在bash运行时覆盖，例如：
~~~bash
python main.py -m train -n resnet34 -l 1e-4 -e 1000 -c 200 -is 224 -bs 16
~~~
表示单折训练，网络模型选择resnet34，初始学习率为1e-4，迭代次数1000，类别数200，图像大小224*224，批次大小64，**这6个参数都是可选参数，在不输入这些参数时，采用config.py中的INIT_TRAINER对应参数的值**。

# 4.模型输出以及查看和修改
 
## 推理过程：最终结果和中间结果输出  

**最终结果输出:**.analysis/result/xxx/foldk/foldk.csv，为第k折交叉验证的真实标签'true'，预测标签'pred'，预测概率'prob'，图像路径'path'；.analysis/result/xxx/foldk_report.csv，为第k折交叉验证的分类报告;.analysis/result/xxx/foldk_fc_weight.npy，为了方便做CAM输出的全连接层的权重。以上csv格式文件使用pandas读取或者直接打开查看；npy格式文件使用numpy读取。

在main.py中搜索'csv_file'，在trainer.py中搜索'result'，可以查看和修改最终结果输出相关代码。

**中间结果输出:**.mid_feature/xxx/foldk/xxx，为逐个输出测试集样本的钩子获取的中间结果，保存为hdf5格式的字典，读取方式为使用.converter/common_util.py的函数:

~~~python
def hdf5_reader(data_path, key=None)
~~~

其中，key=['feature_in', 'feature_out']，分别代表钩子获取的某层的输入和输出特征，利用可视化方法可以对中间结果进行可视化（见5.可解释性）。

在main.py中搜索'save_as_hdf5'，在trainer.py中搜索'feature_in'，可以查看和修改中间结果输出相关代码。

## 训练过程：模型参数和过程数据输出  

**模型参数输出:**.ckpt/xxx/foldk/epoch=xxx.pth，格式为torch.save直接保存的字典（用于加载模型参数），包含key-value对：迭代次数'epoch'，保存路径'save_dir'，模型参数字典'state_dict'，优化器参数字典'optimizer'。读取方式为

~~~python
data = torch.load(weight_path)['key']
~~~

在trainer.py中搜索'saver'，可以查看和修改模型参数输出相关代码。

**过程数据输出:**.log/xxx/foldk/，首先使用tensorboard指定日志目录为log/:

~~~bash
tensorboard --logdir log
~~~

然后访问<http://localhost:6006/>，如果是命令行服务器则需先将6006端口代理到本地，然后本地访问上述网址。

在trainer.py中搜索'writer.add_scalar'，可以查看和修改过程数据输出相关代码。

# 5.可解释性
文件main_interpretability.py用于计算模型对测试图像的注意力热度图，通过分析模型对图片的关注区域可以解释模型是否学习到特定类别信息。使用时需要修改输入的图像和特征路径，两者要对应。
另外还可通过分析分类错误的样本，寻找针对性解决办法，提高性能。

# 6.可进一步提升
1）由于提供的教学服务器GPU显存限制，只能使用较小的模型和较小的batchsize，并使用混合精度训练。如果有更多的计算资源可以尝试从更换模型方面提升效果。

1）可基于进行交叉验证，并降每折模型的测试结果集成（如投票法）提高鲁棒性。

2）探索数据增强或Dropout,BN等以提升模型泛化性。

3）使用辅助损失。（抛砖引玉，如batch内对比损失，与batch内同类别的样本特征表达拉近（余弦相似度或欧氏距离），与不同类别的样本特征表达拉远）。

4）使用VLM文本信息或GAN生成更多图像辅助学习（见7.选做）。


# 7.选做
## VLM介绍及使用
使用CLIP作为VLM模型，论文：https://arxiv.org/pdf/2103.00020v1.pdf ，main_VLM.py示例中使用huggingface的transformers库中的CLIPModel，也可以使用官方CLIP代码：https://github.com/OpenAI/CLIP ，通过引入文本模态信息提升细粒度图像分类效果

## GAN生成图片
构建并训练一个 DCGAN 模型，用于生成类似训练数据的图像；
在模型训练完成后，可以生成合成图像，并探索使用合成数据来提升分类框架的性能。